# Paying the game

In this notebook we'll start looking at how to play the game.

First, we'll load in the work we did on symmetries in the previous notebook.

In [1]:
)copy notebook4

./notebook4.dws saved Thu Dec 10 10:13:18 2020

In [2]:
)fns

encode m r show

`)vars` is another useful system command. It shows us the names of the variables that `)copy` brought in for us.

In [3]:
)vars

board1 boards encoded hereDir i mi mr180 mr270 mr90 r180 r270 r90 start symmetries

In [4]:
⎕io ← 0

## Using symmetries

Let's start by looking at how we can use our work on symmetries to reduce the number of board positions we need to consider when playing.

For any given board position there are up to seven other positions that are equivalent to it.

Why did I say *up to*?

Some board configurations are symmetric, so some transformations will leave them unchanged.

Here's an extreme example. Imagine that `x` has played in the centre of the board.
The board is the same whether you rotate it, reflect it, combine rotations and reflections or leave it unchanged.


In [5]:
 ⊢ symmetric ← 3 3⍴'....×....'

...
.×.
...

To examine the symmetries using APL we need to convert the board to its vector representation.

Let's create a function to do that.

In [6]:
see←{{,'.×○'⍳⍵}⍤2⊣⍵}

It's a little more complicated than you might expect becasue it can handle arrays of boards as well as individual boards.

Let's try it out.

In [11]:
+sym ← see symmetric

0 0 0 0 1 0 0 0 0

You can think of `see` as an inverse to `show`.

Let's check the symmetric transformations on the board.

In [12]:
sym[symmetries]

0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0
0 0 0 0 1 0 0 0 0

They are all identical (as they should be).

How can we get rid of the redundant copies?

APL has a primitive function *Unique* represented as `∪` which does just that.

In [16]:
∪sym[symmetries] ⍝ the unique version

0 0 0 0 1 0 0 0 0

Let's look at a position with some symmetry.

In [22]:
⊢ asymmetric ← 3 3⍴'.×.×○×...'

.×.
×○×
...

In [23]:
+less ← see asymmetric

0 1 0 1 2 1 0 0 0

In [24]:
less[symmetries]

0 1 0 1 2 1 0 0 0
0 1 0 0 2 1 0 1 0
0 0 0 1 2 1 0 1 0
0 1 0 1 2 0 0 1 0
0 1 0 1 2 1 0 0 0
0 1 0 1 2 0 0 1 0
0 0 0 1 2 1 0 1 0
0 1 0 0 2 1 0 1 0

In [25]:
∪less[symmetries]

0 1 0 1 2 1 0 0 0
0 1 0 0 2 1 0 1 0
0 0 0 1 2 1 0 1 0
0 1 0 1 2 0 0 1 0

Now there are four distinct transformations.

Let's see what they are.

In [27]:
⊂⍤2 show ∪less[symmetries]

┌───┬───┬───┬───┐
│.×.│.×.│...│.×.│
│×○×│.○×│×○×│×○.│
│...│.×.│.×.│.×.│
└───┴───┴───┴───┘

In [29]:
show board1

××.
.○○
.×.

In [31]:
⊂⍤2 show ∪board1[symmetries]

┌───┬───┬───┬───┬───┬───┬───┬───┐
│××.│..×│.×.│.○.│.××│×..│.×.│.○.│
│.○○│×○×│○○.│×○×│○○.│×○×│.○○│×○×│
│.×.│.○.│.××│×..│.×.│.○.│××.│..×│
└───┴───┴───┴───┴───┴───┴───┴───┘

`board1` is asymmetric and so it has eight distinct symmetries.

## Canonical representations

In order to apply Michie's first simplification we need a way of converting every board position to *just one* of its symmetries.

We'll use the idea of a *canonical representation*. I've borrowed that term from Mathematics, but the mathematical details don't matter to us.
You'll see how we create and use canonical representations in below.

### Defining a position's canonical representation

As we've seen, every board position is a member of a set of one to eight boards which can be regarded as the same as far as the game is concerned.

The functions `encode` and `decode` allow us to switch between the vector representation of the position and a number in the range zero to 19682.

The symmetries of a given position form a set of up to eight vectors, and each vector can be converted to its unique number using `decode`.
We'll take the smallest of those numbers, convert that to a vector and use that as the canonical representation of the position.

Let's do that for the boards we just considered: `sym`, `less` and `board1`.
 We are going to perform the same operation on each, so we can save time by defining a function to do it.

In [53]:
canonical ← {encode ⌊⌿ decode ⍵[symmetries]}
show canonical sym

...
.×.
...

In [54]:
show canonical less

...
×○×
.×.

In [55]:
show canonical board1

..×
×○×
.○.

Can we find out the canonical representations of all positions?

Let's try

In [58]:
all ← canonical encode ⍳3*9

RANK ERROR
canonical[0] canonical←{encode⌊⌿decode ⍵[symmetries]}
                                        ∧


What's gone wrong?

We've used a simple version of indexing in the definition of canonical.
The `[]` form of indexing corresponds to the notation that is used in a lot of programming languages, but the format is slightly different for vectors and matrices.

Luckily APL has another form of indexing based on  the *Index function* represented as `⌷`.

You can see how it works in the example below.

In [65]:
(⊂0 3 2)⌷7 8 9 10 20

7 10 9

I've called the function we want `ild`, which is short for *index last dimension*.

Here's its definition, along with an updated definition of canonical.

#TODO: add canonical

ild ← {(⊂⍺) ⌷[¯1+⍴⍴⍵]⍵} ⍝ index last dimension
